<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course4/week1/Convolution_model_Application_v1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks: 적용하기 #

코스 4 두 번째 과제에 오신것을 환영합니다. 여러분은,
- Tensorflow 모델을 구현하는데 필요한 여러 helper 함수들을 먼저 구현하고,
- Tensorflow를 사용해 완전히 동작하는 ConvNet 모델을 만듭니다.

**이 과제가 끝난 후 여러분들은**:
- 텐서플로우를 사용해서  분류 문제(classification problem)를 해결하는 ConvNet을 개발하고 훈련할 수 있습니다.

이번 과제는 여러분들이 TensorFlow에 익숙하다는 것을 가정하고 진행합니다. 만약 Tensorflow에 익숙하지 않다면, 코스 2의 3주차 과제("*Improving deep neural networks*") 를 통해서 텐서플로우 튜토리얼을 짚어보고 오세요.

## 1.0 - Tensorflow model ##

지난 과제에서, 여러분들은 합성곱 신경망(convolutional neural networks: CNN) 의 실제 동작 원리를 파악하기 위해 numpy를 사용한 helper 함수를 개발한 바 있습니다. 그러나 오늘날 대부분의 딥 러닝 프로그램들은 쉽게 호출할 수 있는 빌트인 함수들의 묶음을 많이 가지고 있는 프레임워크를 사용해서 개발됩니다.

늘 그랬듯이, 패키지를 받으면서 시작해봅시다.

In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from cnn_utils import *

%matplotlib inline
np.random.seed(1)

다음 코드 블록은 지금부터 훈련시킬 수화 데이터셋을 받아오는 코드입니다.

In [ ]:
# Loading the data (signs)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

이전에도 이 데이터를 보신 적이 있을겁니다. 지금부터 학습시킬 수화 데이터셋은 0부터 5까지의 숫자를 6개의 사인으로 구분한 데이터입니다.

<img src="arts/SIGNS.png" style="width:800px;height:300px;">

다음 코드 블록은 라벨링된 훈련 데이터 이미지를 시각화해서 보여줍니다. 다른 이미지를 보고싶으면 `index` 값을 바꿔서 코드를 재실행해보세요.

In [ ]:
# Example of a picture
index = 6
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

코스 2에서, 여러분은 이미 이 데이터셋에 대해 fully-connected(완전히 연결된) 신경망을 개발한 적이 있습니다. 하지만 이미지 데이터셋에 대해서는 사실 fully-connected 신경망보다는 ConvNet을 사용하는 것이 더 일반적입니다.

시작하기에 앞서, 주어진 데이터셋의 shape를 파악해봅시다.

In [ ]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
conv_layers = {}

## 1-1. Create placeholders ##

Tensorflow에서는 세션을 실행할 때 모델에 투입될 입력 데이터에 대한 placeholder를 만들어야 합니다.

**연습 문제**: 입력 이미지 X와 라벨링된 출력값 Y에 대한 placeholder를 만드는 아래 함수를 구현해보세요. 이를 위해 배치 사이즈를 "None"으로 설정할 수 있습니다. 이 None은 이 값을 나중에 설정할 수 있도록 해 줍니다. 추가로, X와 Y는 각각 아래의 shape를 가지고 있어야 합니다.
- `x.shape` = **(None, n_H0, n_W0, n_C0)**
- `y.shape` = **(None, n_y)**

힌트 : `tf.placeholder` 에 대한 [문서를 참조하세요](https://www.tensorflow.org/api_docs/python/tf/placeholder)

In [ ]:
# GRADED FUNCTION: create_placeholders

def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    ### START CODE HERE ### (≈2 lines)
    X = None
    Y = None
    ### END CODE HERE ###
    
    return X, Y

In [ ]:
X, Y = create_placeholders(64, 64, 3, 6)
print ("X = " + str(X))
print ("Y = " + str(Y))

**모범 답안**

<table> 
<tr>
<td>
    X = Tensor("Placeholder:0", shape=(?, 64, 64, 3), dtype=float32)

</td>
</tr>
<tr>
<td>
    Y = Tensor("Placeholder_1:0", shape=(?, 6), dtype=float32)

</td>
</tr>
</table>

## 1.2 - Initialize parameters ##

`tf.contrib.layers.xavier_initializer(seed=0)` 코드를 사용해서 가중치/필터의 역할을 하는 $W1$과 $W2$를 초기화해봅시다. bias 변수에 대해서는 걱정할 필요가 없습니다. Tensorflow 함수가 bias를 알아서 처리해주기 때문이죠. 또한 `conv2d` 함수에 대한 가중치/필터만 초기화해주면 됩니다. Fully connected 레이어에 대한 가중치는 Tensorflow에서 자동으로 초기화해주는데, 이 부분은 과제 후반부에서 자세히 설명하겠습니다.

**연습 문제** : `initialize_parameters()` 함수를 구현하세요. 각 그룹의 차원은 미리 제공됩니다. (1, 2, 3, 4)의 shape를 가지는 가중치 행렬 $W$를 초기화하기 위해선 아래 코드를 사용해야 합니다.
```python
W = tf.get_variable("W", [1,2,3,4], initializer = ...)
```

**tf.get_variable**
`tf.get_variable`의 [문서에 의하면](https://www.tensorflow.org/api_docs/python/tf/get_variable),
```
Gets an existing variable with these parameters or create a new one.
```

따라서 이 함수를 사용하여 지정된 이름으로 tensorflow 변수를 만들 수 있지만 변수가 이미 존재하는 경우 동일한 이름의 기존 변수를 가져옵니다.

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters():
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]
                        W2 : [2, 2, 8, 16]
    Note that we will hard code the shape values in the function to make the grading simpler.
    Normally, functions should take values as inputs rather than hard coding.
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)                              # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 2 lines of code)
    W1 = None
    W2 = None
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "W2": W2}
    
    return parameters

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess_test:
    parameters = initialize_parameters()
    init = tf.global_variables_initializer()
    sess_test.run(init)
    print("W1[1,1,1] = \n" + str(parameters["W1"].eval()[1,1,1]))
    print("W1.shape: " + str(parameters["W1"].shape))
    print("\n")
    print("W2[1,1,1] = \n" + str(parameters["W2"].eval()[1,1,1]))
    print("W2.shape: " + str(parameters["W2"].shape))

**모범 답안:**

```
W1[1,1,1] = 
[ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
W1.shape: (4, 4, 3, 8)


W2[1,1,1] = 
[-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]
W2.shape: (2, 2, 8, 16)
```

## 1.3 - Forward propagation ##

아래 함수들은 텐서플로우에서, 컨볼루션 연산을 수행하기 위해서 제공되는 빌트인 함수들입니다.

- `tf.nn.conv2d(X, W, strides = [1, s, s, 1], padding = 'SAME')` : 주어진 input $X$와 필터 그룹 $W$가 주어지면 이 함수는 X에 대해서 W 필터들을 컨볼루션합니다. 세 번째 매개변수인 `strides = [1, s, s, 1]`는 입력의 각 차원 (m, n_H_prev, n_W_prev, n_C_prev)에 대한 stride 값을 나타냅니다. 일반적으로 예제 수(배열의 첫 값 1)와 채널의 수(배열의 네 번째 값 1)는 stride 1로 선택하므로 [1, s, s, 1]을 사용합니다. [링크](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)에서 conv2d에 대한 전체 문서를 읽을 수 있습니다. 

- `tf.nn.max_pool(A, ksize=[1,f,f,1], strides=[1,s,s,1], padding='SAME')` : 이 함수는 주어진 입력 A에 대해서 (f,f)의 사이즈를 가진 창을 (s, s)의 stride로 max pooling 연산한 결과값을 리턴합니다. 주로 한번에 한 개의 샘플 데이터를 연산하고, 채널도 1개만 사용할 것이므로 `ksize=[1,f,f,1]`로 선언합니다. 더 많은 정보는 [링크](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool)를 참조하세요.

- `tf.nn.relu(Z)`: 
Z의 ReLU (어떤 shape도 가능)를 계산합니다. [relu api 문서](https://www.tensorflow.org/api_docs/python/tf/nn/relu) 에서 전체 문서를 읽을 수 있습니다.

- `tf.contrib.layers.flatten(P)` : 주어진 텐서 "P"에 대하여, 이 함수는 배치의 각 훈련 예제를 가져와서 1차원 벡터로 평면화합니다.
  - 만약 텐서 P가 (m, h, w, c)의 shape를 가지고 있다면, 이 함수는 (batch_size, k)의 shape를 가진 평면화된 텐서를 리턴합니다. 여기서 k는 $k = h \times w \times c$ 입니다. 
    - m : 배치 사이즈
    - h : 높이
    - w : 너비
    - c : 채널의 개수
  - 예를 들어, [100, 2, 3, 4] 차원의 텐서에 대해서 이 함수는 [100, 24]의 평면화된 텐서를 리턴합니다. [flatten 문서](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/flatten) 에서 전체 내용을 확인할 수 있습니다.
  
- `tf.contrib.layers.fully_connected(F, num_outputs)`: 주어진 평면 입력값 F에 대해서 fully connected 레이어를 계산한 결과를 리턴합니다. 더 많은 정보는 [full_connected 문서](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/fully_connected) 에서 확인할 수 있습니다.


마지막 `tf.contrib.layers.full_connected` 함수에서, fully connected 레이어는 자동적으로 가중치를 초기화하고 모델이 훈련되는 과정에서 자동으로 학습합니다. 따라서 별도로 파라미터를 초기화해줄 필요가 없는 것입니다.

#### Window, Kernal, filter ####
"window", "kernal", "filter" 라는 단어들은 결국 같은 말입니다. 이것이 `ksize` (kernal size)와 `(f,f`(filter size)와 같은 표기법을 사용하는 이유입니다. "kernal"과 "filter" 모두 컨볼루션 연산에서 사용하는 창을 의미합니다.

**연습 문제**
`forward_propagation()` 함수를 구현하세요. 모델 구조는 `CONV2D -> RELU -> MAXPOOL -> CONV2D -> MAXPOOL -> FLATTEN -> FULLYCONNECTED` 으로 이루어져 있습니다. 위의 함수 설명을 참고하세요.

구체적으로는, 아래의 단계에 맞게 구현해야 합니다.

- CONV2D: stride 1, padding "SAME"
- RELU
- MAXPOOL: (8x8) filter size, stride 8, padding "SAME"
- CONV2D: stride 1, padding "SAME"
- MAXPOOL : (4x4) filter size, stride 4, padding "SAME"
- FLATTEN
- FULLYCONNECTED(FC) 레이어 : 
비선형 활성화 함수없이 fully connected 레이어를 적용합니다. 여기서 softmax를 부르지 마세요. 그러면 출력 레이어에 6 개의 뉴런이 생성되고 나중에 소프트 맥스로 전달됩니다. TensorFlow에서 softmax와 cost 함수는 하나의 함수로 묶여 있으며 비용을 계산할 때 다른 함수에서 호출합니다.

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Note that for simplicity and grading purposes, we'll hard-code some values
    such as the stride and kernel (filter) sizes. 
    Normally, functions should take these values as function parameters.
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    ### START CODE HERE ###
    # CONV2D: stride of 1, padding 'SAME'
    Z1 = None
    # RELU
    A1 = None
    # MAXPOOL: window 8x8, stride 8, padding 'SAME'
    P1 = None
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z2 = None
    # RELU
    A2 = None
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P2 = None
    # FLATTEN
    F = None
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    Z3 = None
    ### END CODE HERE ###

    return Z3

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    X, Y = create_placeholders(64, 64, 3, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(Z3, {X: np.random.randn(2,64,64,3), Y: np.random.randn(2,6)})
    print("Z3 = \n" + str(a))

**모범 답안**:

```
Z3 = 
[[-0.44670227 -1.57208765 -1.53049231 -2.31013036 -1.29104376  0.46852064]
 [-0.17601591 -1.57972014 -1.4737016  -2.61672091 -1.00810647  0.5747785 ]]
```

## 1.4 - Compute cost ##

비용을 계산하는 아래 함수를 구현해보세요. 비용함수는 인공신경망의 예측 결과가 라벨링된 실제 정답과 얼마나 차이가 있는지를 계산합니다. 가중치를 조정하면서 비용을 최소화하면, 인공신경망의 예측 능력을 발전시킬 수 있습니다.

아래 두 함수를 사용하세요.

- `tf.nn.softmax_cross_entropy_with_logits(logits=Z, labels=Y)` : softmax 엔트로피 손실을 계산합니다. 이 함수는 
그의 함수는 소프트 맥스 활성화 함수와 결과 손실을 모두 계산합니다. [이 문서에서] (https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits) 자세한 내용을 확인할 수 있습니다.

- `tf.reduce_mean`: 은 텐서 차원에서 모든 원소의 평균을 계산합니다. 이를 사용하여 전체 cost를 얻기 위해 모든 데이터에 대한 손실 합계를 계산하십시오.[이 문서에서] (https://www.tensorflow.org/api_docs/python/tf/reduce_mean) 더 자세한 내용을 확인할 수 있습니다.


#### softmax_cross_entropy_with_logits 에 대한 세부사항 ####
- Softmax 함수는 분류 알고리즘에 사용할 수 있도록 출력 형식을 지정하는데 사용됩니다. 각 카테고리에 대해 0과 1사이의 값을 할당합니다. 여기서 모든 카테고리의 예측 결과값의 합은 1입니다.
- 크로스 엔트로피는 모델의 예측된 분류 결과를 실제 라벨과 비교하여 모델 예측의 손실을 나타내는 수를 계산합니다.
- "Logit"은 가중치를 곱하고 bias를 더한 결과입니다. Logit은 relu와 같은 활성화 함수를 통해 전달되며, 그 결과를 activation 이라고 합니다.
- 이 함수는 활성화 값이 아닌 `logit`을 입력으로 받는 `softmax_cross_entropy_with_logits` 함수입니다. 모델을 사용하여 softmax로 예측한 다음, 크로스 엔트로피를 사용해 예측을 실제 라벨과 비교합니다. 이는 계산을 최적화하는 단일 기능으로 작동합니다.

In [ ]:
# GRADED FUNCTION: compute_cost 

def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (number of examples, 6)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    ### START CODE HERE ### (1 line of code)
    cost = None
    ### END CODE HERE ###
    
    return cost

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    X, Y = create_placeholders(64, 64, 3, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(cost, {X: np.random.randn(4,64,64,3), Y: np.random.randn(4,6)})
    print("cost = " + str(a))

**모범 답안**: 
```
cost = 2.91034
```

## 1.5 Model ##

최종적으로 앞서 구현한 helper 함수들을 합쳐서 모델을 만들어봅시다. 이후 수화 데이터셋으로 모델을 훈련시켜보겠습니다.

**연습문제** : 아래 함수를 완성하세요.

모델은 다음의 함수를 포함해야 합니다.
- create placeholders
- initialize parameters
- forward propagate
- compute the cost
- create an optimizer

이후 session을 만들고 `num_epochs` 만큼 반복하여 훈련시켜보세요. 훈련은 `mini-batches` 단위로 수행합니다. 각 미니 배치마다 최적화 함수가 들어가야합니다. 다음은 [초기화를 위한 힌트입니다](https://www.tensorflow.org/api_docs/python/tf/global_variables_initializer).

#### Adam Optimizer ####

`tf.train.AdamOptimizer(learning_rate=...)` 함수를 사용하면 최적화 함수를 만들 수 있습니다. 이 최적화 인스턴스는 `minimize(loss=...)`라는 함수를 가지고 있어서, 최소화시켜야할 비용 함수를 파라미터로 넣고 호출하면 됩니다.

더 자세한 사항은 [Adam Optimizer 함수에 대한 문서](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) 를 살펴보세요.

#### Random mini batches ####

코스 2를 들었다면, `random_mini_batches()` "Optimization" 파트의 프로그래밍 과제에서 위 함수를 구현한 바 있을겁니다. 이 함수는 미니 배치의 리스트를 리턴합니다. 이번 과제에서는 `cnn_utils.py` 파일에 미리 구현된 함수를 제공합니다. 아래와 같이 호출하세요.
```python
minibatches = random_mini_batches(X, Y, mini_batch_size=64, seed=0)
```
(코드에서 사용할 때 올바른 변수 이름을 선택하는 것이 좋습니다).

#### Evaluating the optimizer and cost ####

루프 내에서 각 미니 배치에 대해`tf.Session` 객체 (`sess`라고 함)를 사용하여 입력과 라벨의 미니 배치를 신경망에 공급하고 최적화 프로그램에 대한 텐서 및 비용을 평가합니다. 우리는 (계산) 그래프 데이터 구조를 구축했으며 최적화 프로그램과 비용에 대한 값을 얻기 위해 입력과 라벨링 데이터를 제공하고`sess.run ()`을 사용해야합니다.

아래와 같은 방식으로 사용하세요.
```
output_for_var1, output_for_var2 = sess.run(
  fetches=[var1, var2],
  feed_dict={
    var_inputs: the_batch_of_inputs,
    var_labels: the_batch_of_labels
  }
)
```

* `sess.run`은 평가할 객체 목록으로 첫 번째 인수`fetches`를 사용합니다 (이 경우 최적화 함수와 비용을 평가하려고합니다).
* `feed_dict`: 파라미터에에 대한 딕셔너리도 필요합니다.
* Key는 위의`create_placeholders` 함수에서 생성 한 `tf.placeholder` 변수입니다.
* Value는 각 미니 배치에 대한 실제 numpy 배열을 보유하는 변수입니다.
* sess.run은`fetches`에 주어진 목록과 동일한 순서로 평가 된 텐서의 튜플을 출력합니다.

sess.run 사용 방법에 대한 자세한 내용은 문서 [tf.Sesssion # run](https://www.tensorflow.org/api_docs/python/tf/Session#run) 문서를 참조하세요.

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.009,
          num_epochs = 100, minibatch_size = 64, print_cost = True):
    """
    Implements a three-layer ConvNet in Tensorflow:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X_train -- training set, of shape (None, 64, 64, 3)
    Y_train -- test set, of shape (None, n_y = 6)
    X_test -- training set, of shape (None, 64, 64, 3)
    Y_test -- test set, of shape (None, n_y = 6)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    train_accuracy -- real number, accuracy on the train set (X_train)
    test_accuracy -- real number, testing accuracy on the test set (X_test)
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of the correct shape
    ### START CODE HERE ### (1 line)
    X, Y = None
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = None
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = None
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = None
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    ### START CODE HERE ### (1 line)
    optimizer = None
    ### END CODE HERE ###
    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
     
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                """
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost.
                # The feedict should contain a minibatch for (X,Y).
                """
                ### START CODE HERE ### (1 line)
                _ , temp_cost = None
                ### END CODE HERE ###
                
                minibatch_cost += temp_cost / num_minibatches
                

            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # Calculate the correct predictions
        predict_op = tf.argmax(Z3, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
                
        return train_accuracy, test_accuracy, parameters

다음 셀을 실행하여 100 세대 동안 모델을 훈련시킵니다. epoch 0과 5 이후의 비용이 우리의 출력과 일치하는지 확인하십시오. 그렇지 않은 경우 셀을 중지하고 코드로 돌아갑니다!

In [ ]:
_, _, parameters = model(X_train, Y_train, X_test, Y_test)

**모범 답안** : 완벽하게 일치하지 않을 수 있지만 여러분의 output은 모범 답안의 예시와 비슷해야 하고, cost 값이 감소해야합니다.

<table> 
<tr>
    <td> 
    Cost after epoch 0 =
    </td>
    <td> 
      1.917929
    </td> 
</tr>
<tr>
    <td> 
    Cost after epoch 5 =
    </td>
    <td> 
      1.506757
    </td> 
</tr>
<tr>
    <td> 
    Train Accuracy   =
    </td>
    <td> 
      0.940741
    </td> 
</tr> 
<tr>
    <td> 
    Test Accuracy   =
    </td>
    <td> 
      0.783333
    </td> 
</tr> 
</table>

축하합니다! 과제를 완료하고 테스트 세트에서 거의 80 % 정확도로 수화 언어를 인식하는 모델을 구축했습니다. 원하는 경우이 데이터 세트를 자유롭게 사용해보세요. 하이퍼 파라미터를 조정하는 데 더 많은 시간을 소비하거나 정규화를 사용하여 실제로 정확도를 향상시킬 수 있습니다 (이 모델은 분명히 높은 분산을 가짐).

In [ ]:
fname = "images/thumbs_up.jpg"
image = np.array(ndimage.imread(fname, flatten=False))
my_image = scipy.misc.imresize(image, size=(64,64))
plt.imshow(my_image)